# parent_selection.py
Determines how to select parents based off of the selection type entered 

In [1]:
import constraints
import numpy
from numpy.random import seed
from numpy.random import randint

Driver of the module, selects a number of parents using fitness scores and determines what method of selection to use by what the config file says.  Reads global variable to determine selection type, then calls sub-function. Returns a list containing indexes of selected parents. 

In [2]:
def select_parents(num_parents, fitnesses):
    parent_list = [0] * num_parents
    if constraints.parent_selection == "random":
        return select_parents_random(num_parents, fitnesses)
    if constraints.parent_selection == "roulette":
        return select_parents_roulette(num_parents, fitnesses)
    if constraints.parent_selection == "rank":
        return select_parents_rank(num_parents, fitnesses)
    for parent in range(num_parents):
        if constraints.parent_selection == "tournament":
            parent_list[parent] = select_parents_tournament(constraints.tournament_y, fitnesses)
    return parent_list

Selects parents based off of a tournament style: A subset of size y_parents is created consisting of randomly chosen parents, and the index of the parent with the max fitness is returned.

In [3]:
def select_parents_tournament(y_parents, fitnesses):
    parent_sublist = [0] * y_parents
    index = 0
    for i in range(y_parents):
        parent_sublist[i] = randint(0,len(fitnesses))
    max = 0
    for j in parent_sublist:
        if fitnesses[j] > max:
            max = fitnesses[j]
            index = j
    return index

Selects parents randomly based off of index positions. Does not allow for duplicate parent values. Returns a list of indexes of the randomly selected parents.

In [4]:
def select_parents_random(num_parents, fitnesses):
    parents = []
    for i in range(num_parents):
        rand_parent = randint(0, len(fitnesses))
        if rand_parent not in parents:
            parents.append(rand_parent)
            break 
    return parents

Selects parents based on a "roulette-wheel" style: the fitness list is sorted then a random integer between 0 and sum(fitnesses) if selected (roulette_value). The fitness list is iterated over and each fitness is added to the partial sum counter, while counter < roulette_value. When this constraint is violated, the index of the parent that violated it is added to the parent sublist. NOTE: this method allows for duplicate parents

In [5]:
def select_parents_roulette(num_parents, fitness):
    parent_sublist = [] 
    sum_fitnesses = sum(fitness)
    fitness_dict = {}
    for position, value in enumerate(fitness):
        fitness_dict[position] = value
    sorted_fitness = {k: v for k, v in sorted(fitness_dict.items(), key=lambda item: item[1])}
    while len(parent_sublist) < num_parents:
        counter = 0
        roulette_value = randint(0, sum_fitnesses)
        for original_index, fitness in sorted_fitness.items():
            counter += fitness
            if counter < roulette_value:
                continue
            else:
                parent_sublist.append(original_index)
                break
    return parent_sublist

Selects parents by ranking them in order of fitness and selecting the top n parents where n = number of parents required.

In [6]:
def select_parents_rank(num_parents, fitness):
    parent_sublist = [] 
    fitness_dict = {}
    for position, value in enumerate(fitness):
        fitness_dict[position] = value
    sorted_fitness = {k: v for k, v in sorted(fitness_dict.items(), key=lambda item: item[1], reverse=True)}
    sorted_fitness_list = list(sorted_fitness)
    for pair in range(0, num_parents):
        parent_sublist.append(sorted_fitness_list[pair])
    return parent_sublist